In [1]:
#!pip install -q gradio openai pypdf tiktoken langchain

import os
os.environ["OPENAI_API_KEY"] = "Your_OpenAI_API_Key"

import gradio as gr
from langchain import OpenAI, PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from PyPDF2 import PdfReader

# Initialize LLM
llm = OpenAI(temperature=0, max_tokens=2000)

def generate_insights(sector, description):
    prompt = (
        f"You are an AI advisor assisting an aspiring entrepreneur working in the {sector} sector. "
        f"They have described their product or solution as: {description}. "
        f"Your task is to provide in-depth insights on potential competitors, market challenges, and key steps to refine their idea. "
        f"Offer a strategic roadmap that helps them gain confidence and move forward."
    )
    response = llm.generate([prompt])
    return response.dict()['generations'][0][0]['text']

def generate_startup_pitch(sector, description):
    prompt = (
        f"You are an AI mentor helping an aspiring entrepreneur craft a compelling pitch for their new idea in the {sector} sector. "
        f"Their concept is: {description}. "
        f"Create a pitch that clearly states the unique value proposition, showcases market potential, and encourages early supporters or investors to believe in their vision."
    )
    response = llm.generate([prompt])
    return response.dict()['generations'][0][0]['text']

def perform_domain_analysis(sector, description):
    prompt = (
        f"You are an AI consultant working with an aspiring entrepreneur who has an idea in the {sector} sector. "
        f"They've explained: {description}. "
        f"Provide a domain analysis identifying current trends, market gaps, key opportunities, and potential risks, helping them feel more confident about the domain they are entering."
    )
    response = llm.generate([prompt])
    return response.dict()['generations'][0][0]['text']

sectors = [
    "Engineering", "Agro Chemicals", "Air Transport Service", 
    "Auto Ancillaries", "Automobile", "Banks", "Bearings", "Cables",
    "Capital Goods - Electrical Equipment", "Capital Goods-Non Electrical Equipment",
    "Castings, Forgings & Fastners", "Cement", "Cement - Products", "Ceramic Products",
    "Chemicals", "Computer Education", "Construction", "Consumer Durables",
    "Credit Rating Agencies", "Crude Oil & Natural Gas", "Diamond, Gems and Jewellery",
    "Diversified", "Dry cells", "E-Commerce/App based Aggregator", "Edible Oil",
    "Education", "Electronics", "Aerospace & Defence", "Entertainment", "Ferro Alloys",
    "Fertilizers", "Finance", "Financial Services", "FMCG", "Gas Distribution",
    "Glass & Glass Products", "Healthcare", "Hotels & Restaurants",
    "Infrastructure Developers & Operators", "Infrastructure Investment Trusts",
    "Insurance", "IT - Hardware", "IT - Software", "Leather", "Logistics",
    "Marine Port & Services", "Media - Print/Television/Radio", "Mining & Mineral products",
    "Miscellaneous", "Non Ferrous Metals", "Oil Drill/Allied", "Packaging",
    "Paints/Varnish", "Paper", "Petrochemicals", "Pharmaceuticals",
    "Plantation & Plantation Products", "Plastic products", "Plywood Boards/Laminates",
    "Power Generation & Distribution", "Power Infrastructure", "Printing & Stationery",
    "Quick Service Restaurant", "Railways", "Readymade Garments/ Apparells",
    "Real Estate Investment Trusts", "Realty", "Refineries", "Refractories", "Retail",
    "Ship Building", "Shipping", "Steel", "Stock/ Commodity Brokers", "Sugar",
    "Telecom-Handsets/Mobile", "Telecomm Equipment & Infra Services", "Telecomm-Service",
    "Textiles", "Tobacco Products", "Trading", "Tyres", "Other"
]

def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len)
    return text_splitter.split_text(text)

def get_vectorstore(text_chunks):
    embeddings = OpenAIEmbeddings()
    return FAISS.from_texts(texts=text_chunks, embedding=embeddings)

def get_conversation_chain(vectorstore):
    llm = ChatOpenAI(temperature=0)
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    return ConversationalRetrievalChain.from_llm(llm=llm, retriever=vectorstore.as_retriever(), memory=memory)

def handle_userinput(user_question, pdf_docs):
    if not pdf_docs:
        return "Please upload PDFs to proceed."
    raw_text = get_pdf_text(pdf_docs)
    text_chunks = get_text_chunks(raw_text)
    vectorstore = get_vectorstore(text_chunks)
    conversation_chain = get_conversation_chain(vectorstore)
    response = conversation_chain({'question': user_question})
    chat_history = response['chat_history']
    formatted_history = [f"{'User' if i % 2 == 0 else 'AI Assistant'}: {message.content}" for i, message in enumerate(chat_history)]
    return "\n".join(formatted_history)

def summarize_pdf(pdf_docs, custom_prompt=""):
    if not pdf_docs:
        return "Please upload PDFs to proceed."
    summaries = []
    for pdf in pdf_docs:
        loader = PyPDFLoader(pdf)
        docs = loader.load_and_split()
        chain = load_summarize_chain(llm, chain_type="map_reduce")
        summary = chain.run(docs)
        if custom_prompt:
            prompt_template = custom_prompt + """
            {text}
            SUMMARY:"""
            PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
            chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt=PROMPT, combine_prompt=PROMPT)
            custom_summary = chain({"input_documents": docs}, return_only_outputs=True)["output_text"]
        else:
            custom_summary = ""
        summaries.append((summary, custom_summary))
    all_summaries = []
    for i, (base_summary, custom_summary) in enumerate(summaries):
        final_output = f"PDF {i+1} Summary:\n{base_summary.strip()}\n"
        if custom_summary.strip():
            final_output += f"\nCustom Summary:\n{custom_summary.strip()}\n"
        all_summaries.append(final_output)
    return "\n".join(all_summaries)

with gr.Blocks(theme='freddyaboulton/dracula_revamped') as interface:
    gr.Markdown("## Idea Empowerment & Investment Guidance Platform")
    gr.Markdown("""
    Welcome to your all-in-one assistant for nurturing new ideas and supporting confident decision-making.
    This platform offers two main areas:
    - **Founder Confidence Tools**: For aspiring entrepreneurs looking to refine their ideas, understand their market, and pitch with clarity.
    - **Investor Decision Support**: For investors who want a diligent, AI-driven partner to summarize and explore documents, reducing uncertainty in the investment decision process.
    """)

    with gr.Tab("Founder Confidence Tools"):
        gr.Markdown("### Step 1: Define Your Idea")
        gr.Markdown("Provide details about your startup concept. This information will be used across all tools below.")
        with gr.Row():
            selected_sector = gr.Dropdown(label="Select Your Startup's Sector", choices=sectors)
            startup_description = gr.Textbox(label="Describe Your Idea or Product", placeholder="Explain what your startup does and what problem it solves.", lines=3)
        
        gr.Markdown("### Explore Tools to Gain Confidence")
        with gr.Tab("Craft Your Pitch"):
            gr.Markdown("Generate a compelling, investor-ready pitch to communicate your idea's value and potential.")
            pitch_output = gr.Textbox(label="Your Pitch", lines=10)
            submit_pitch_button = gr.Button("Generate Pitch")
            submit_pitch_button.click(generate_startup_pitch, inputs=[selected_sector, startup_description], outputs=[pitch_output])

        with gr.Tab("Gain Market Insights"):
            gr.Markdown("Receive insights on competitors, challenges, and strategic steps, helping you build confidence in your venture.")
            insights_output = gr.Textbox(label="Market Insights", lines=10)
            submit_insights_button = gr.Button("Get Insights")
            submit_insights_button.click(generate_insights, inputs=[selected_sector, startup_description], outputs=[insights_output])

        with gr.Tab("Domain Analysis"):
            gr.Markdown("Understand the current landscape and opportunities in your chosen domain.")
            domain_output = gr.Textbox(label="Domain Analysis", lines=10)
            submit_domain_button = gr.Button("Perform Domain Analysis")
            submit_domain_button.click(perform_domain_analysis, inputs=[selected_sector, startup_description], outputs=[domain_output])

    

    with gr.Tab("Investor Decision Support"):
        gr.Markdown("### Upload Relevant Documents")
        gr.Markdown("Whether it's a startup's pitch deck, business plan, or financial data, upload PDFs here to get summaries and clarifications.")

        with gr.Tab("Ask the PDF"):
            gr.Markdown("#### Interrogate the Documents")
            gr.Markdown("Ask specific questions about the uploaded files to clarify details and reduce uncertainty.")
            pdf_docs_ask = gr.File(label="Upload your PDFs here", file_count="multiple", type="filepath")
            user_question = gr.Textbox(label="Ask a Question")
            ask_output = gr.Textbox(label="AI Assistant's Response", interactive=False, lines=10)
            ask_pdf_btn = gr.Button("Ask")
            ask_pdf_btn.click(handle_userinput, inputs=[user_question, pdf_docs_ask], outputs=[ask_output])

        with gr.Tab("Summarize PDF"):
            gr.Markdown("#### Get Condensed Summaries")
            gr.Markdown("Generate a summary of the entire document set, or provide a custom prompt for a specialized summary.")
            pdf_docs_summarize = gr.File(label="Upload your PDFs here", file_count="multiple", type="filepath")
            custom_prompt_input = gr.Textbox(label="Optional Custom Prompt", placeholder="E.g., 'Focus on financial metrics and growth projections.'", lines=4)
            summary_output = gr.Textbox(label="Summary Output", interactive=False, lines=10)
            summarize_btn = gr.Button("Summarize")
            summarize_btn.click(summarize_pdf, inputs=[pdf_docs_summarize, custom_prompt_input], outputs=[summary_output])

interface.launch(share=True)

C:\Users\IT Department\AppData\Local\Temp\ipykernel_35500\2396091103.py:19: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0, max_tokens=2000)


* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://3f92cc94d249a6968a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
